In [1]:
import os, random
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1'
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm

import torch
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

from model import *
from utils import *
from model import Seq2Seq

import warnings
warnings.filterwarnings(action='ignore')

# Pytorch determistic
random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

In [2]:
def split_behavior(df, behavior):
    behavior_index_value = df[df['0_behavior'] == behavior].index.values
    idx_for_behavior = np.where(behavior_index_value[1:] - behavior_index_value[:-1] > 1)[0] + 1
    return idx_for_behavior

def get_scenario(data):   
    prepare_idx  = split_behavior(data, 'prepare')
    takeoff_idx  = split_behavior(data, 'takeoff')
    forward_idx  = split_behavior(data, 'forward')
    backward_idx = split_behavior(data, 'backward')
    right_idx    = split_behavior(data, 'right')
    left_idx    = split_behavior(data, 'left')
    up_idx       = split_behavior(data, 'up')
    down_idx     = split_behavior(data, 'down')
    land_idx     = split_behavior(data, 'land')
    behavior_idx = [prepare_idx, takeoff_idx, forward_idx, backward_idx, right_idx, left_idx, up_idx, down_idx, land_idx] 
    scenarios = [np.sum(np.asarray(behavior_idx)[:, each_idx]) for each_idx in range(len(prepare_idx))]
    scenarios.append(len(data))
    
    scenario_list = [None for _ in range(len(data))]
    start = 0
    for scenario_num, scenario_idx in enumerate(scenarios):
        for i in range(start, scenario_idx):
            scenario_list[i] = scenario_num
        start = scenario_idx        
    return scenario_list, scenarios

def get_subsystems(all_columns, target_colum):
    system_cate = [each.split('.')[0] for each in all_columns]
    selected_subsystem = all_columns[np.array(system_cate) == target_colum]
    return list(selected_subsystem) + ['timestamp']

def normalize_v1(data, ison_list, train=50, val=5, test=10):
    np.random.seed(random_seed)
    random_scenario = np.random.permutation(np.unique(data['scenario']))
    
    print(random_scenario)
    
    train_scenario = random_scenario[:train]
    val_scenario   = random_scenario[train:train+val]
    test_scenario  = random_scenario[train+val:train+val+test]
    
    train_scaler   = []
    val_scaler   = []
    test_scaler   = []
    
    train_normalized = []
    val_normalized   = []
    test_normalized  = []
    
    for i in np.unique(data['scenario']):
        scaler          = CustomScaler()
        target_data     = data[data['scenario'] == i].drop(['scenario'], axis=1)
        not_norm_column = ison_list.copy()
        not_norm_column.append('0_behavior')
        target_column   = set(target_data.keys()).difference(set(not_norm_column))
        
        if i in train_scenario:
            train_normalized.append(scaler.fit_transform(target_data, target_column))
            train_scaler.append(scaler)
        elif i in val_scenario:
            val_normalized.append(scaler.fit_transform(target_data, target_column))
            val_scaler.append(scaler)
        elif i in test_scenario:
            test_normalized.append(scaler.fit_transform(target_data, target_column))
            test_scaler.append(scaler)
    
    train_norm = np.concatenate(train_normalized)
    val_norm = np.concatenate(val_normalized)
    test_norm = np.concatenate(test_normalized)

    return [train_norm, train_scenario, train_scaler],[val_norm, val_scenario, val_scaler],[test_norm, test_scenario, test_scaler]

def normalize_v2(data, ison_list, train=50, val=5, test=10):
    np.random.seed(random_seed)
    random_scenario = np.random.permutation(np.unique(data['scenario']))
    
    train_scenario = random_scenario[:train]
    val_scenario   = random_scenario[train:train+val]
    test_scenario  = random_scenario[train+val:train+val+test]
    
    np.random.seed(random_seed)
    random_scenario = np.random.permutation(np.unique(data['scenario']))

    train_scenario = random_scenario[:50]
    val_scenario   = random_scenario[50:55]
    test_scenario  = random_scenario[55:65]

    train_idx  = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in train_scenario]
    train_data = data.iloc[sorted(np.hstack(train_idx))]

    val_idx  = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in val_scenario]
    val_data = data.iloc[sorted(np.hstack(val_idx))]

    test_idx  = [np.where(data['scenario'] == each_scenario)[0] for each_scenario in test_scenario]
    test_data = data.iloc[sorted(np.hstack(test_idx))]

    not_norm_column = ison_list.copy()
    not_norm_column.append('0_behavior')
    not_norm_column.append('scenario')
    target_column   = set(train_data.keys()).difference(set(not_norm_column))

    scaler = CustomScaler()
    scaler.fit_transform(train_data, target_column)
    scaler.transform(val_data, target_column)
    scaler.transform(test_data, target_column)
    
    return [train_data, val_data, test_data],\
    [train_scenario, val_scenario, test_scenario],\
    scaler

In [3]:
data = pd.read_csv('px4_normal_nanprocessed.csv')
list_of_scenario, scenario_idx = get_scenario(data)
data['scenario'] = list_of_scenario

# 기타변수 제거
ignore_colums = ['0_environment', '0_target', 'name', 'time', 'timestamp']

# time 변수 위치 파악 및 제거
columns = list(data.keys())
time_list = [each for each in list(data.keys()) if each.find('time')>0]
data.drop(ignore_colums+time_list, axis=1, inplace=True)

# ison 변수 위치 파악
ison_list = [each for each in list(data.keys()) if each.find('ison')>0]
ison_dict = {key: np.where(np.array(list(data.keys())) == key)[0][0] for key in ison_list}

# Behavior 카테고리 변환
behaviors = ['prepare', 'takeoff', 'forward', 'backward', 'up', 'down', 'right', 'left', 'land']
label_dict = Vocabulary()
for each_behavior in behaviors:
    label_dict.add_word(each_behavior)
data['0_behavior'] = data['0_behavior'].map(label_dict.word2idx)

(train_data, val_data, test_data), (train_scenario, val_scenario, test_scenario), scaler = normalize_v2(data, ison_list)

# 시나리오별 성능 확인용
train_scenario_index = train_data['scenario']
val_scenario_index   = val_data['scenario']
test_scenario_index  = test_data['scenario']

X_train, y_train = split_series_v2(train_data.drop(['scenario'], axis=1).values, ison_dict, 10, 1)
X_val, y_val     = split_series_v2(val_data.drop(['scenario'], axis=1).values, ison_dict, 10, 1)
X_test, y_test   = split_series_v2(test_data.drop(['scenario'], axis=1).values, ison_dict, 10, 1)

X_train_drop = X_train[:, :, 1:]
X_val_drop   = X_val[:, :, 1:]
X_test_drop  = X_test[:, :, 1:]

In [4]:
print('Training')
print(f'Scenario: {train_scenario}')
print(f'Shape of the traninig sample for input: {X_train_drop.shape}')
print(f'Shape of the traninig sample for label: {y_train.shape}')

print('\nValidation')
print(f'Scenario: {val_scenario}')
print(f'Shape of the traninig sample for input: {X_val_drop.shape}')
print(f'Shape of the traninig sample for label: {y_val.shape}')

print('\nTest')
print(f'Scenario: {test_scenario}')
print(f'Shape of the traninig sample for input: {X_test_drop.shape}')
print(f'Shape of the traninig sample for label: {y_test.shape}')

Training
Scenario: [53 60  0 45  5 61 16 12 64 30 33  9 41 13 34 25  3 17 40  8  6 63  4 48
 36 19 55 58 56 50 31 44 62 15 27 26 24 47 11 32 59 46 37 29 43 49  1 21
  2 52]
Shape of the traninig sample for input: (22948, 10, 181)
Shape of the traninig sample for label: (22948, 1, 162)

Validation
Scenario: [39 35 23 54 10]
Shape of the traninig sample for input: (2281, 10, 181)
Shape of the traninig sample for label: (2281, 1, 162)

Test
Scenario: [22 18 57 38 20  7 42 14 28 51]
Shape of the traninig sample for input: (4574, 10, 181)
Shape of the traninig sample for label: (4574, 1, 162)


In [5]:
trainSet = CustomDataset(X_train_drop, y_train, flatten=True, which='both')
valSet   = CustomDataset(X_val_drop, y_val, flatten=True, which='both')
testSet  = CustomDataset(X_test_drop, y_test)

train_dataloader = torch.utils.data.DataLoader(trainSet, batch_size=64, shuffle=True, drop_last=True, num_workers=2, pin_memory=True)
val_dataloader   = torch.utils.data.DataLoader(valSet, batch_size=64, shuffle=False, drop_last=False, num_workers=2, pin_memory=True)

In [6]:
IN_FEAT     = X_train_drop.shape[-1]
OUT_FEAT    = 170 # y_train.shape[-1] + 8  # number of classes
IN_SEQ      = 10
OUT_SEQ     = 1
IS_PARALLEL = True
EPOCH       = 10
SAVE_PATH   = 'weights/DNN/'

model = DNN(input_size = IN_FEAT*IN_SEQ, hidden_size = 128, output_size = OUT_FEAT*OUT_SEQ)
#model = Seq2Seq(input_dim=IN_FEAT*IN_SEQ, output_dim=OUT_FEAT)

if IS_PARALLEL:
    model = nn.DataParallel(model)

scaler = torch.cuda.amp.GradScaler()
model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterionMSE = nn.MSELoss()
criterionCE = nn.CrossEntropyLoss()

In [7]:
res = {
    'train_mse':[],
    'train_ce':[],
    'train_total':[],
    
    'val_mse':[],
    'val_ce':[],
    'val_total':[],
    'val_acc':[],
    
    'best_total':[],
}

best_total = np.inf
for epoch in range(EPOCH):

    # Training Procedure
    train_running_step = len(train_dataloader)
    print(f'runnnig step: {train_running_step}')

    model.train();
    train_mse   = 0
    train_ce    = 0
    train_total = 0
    for train_idx, (inputs, labels) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        inputs = inputs.cuda()
        #labels = labels.cuda()
        labels_state = labels[:, 1:].cuda()
        labels_behavior = labels[:, 0].type(torch.LongTensor).cuda()

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(inputs)
            loss_mse = criterionMSE(outputs[:, :-9], labels_state)     # 기체 상태 MSE 예측
            loss_ce = criterionCE(outputs[:, -9:], labels_behavior)    # behavior 확률 변수
            loss = loss_mse + loss_ce
        loss.backward()
        optimizer.step()

        train_mse += loss_mse.item()
        train_ce += loss_ce.item()
        train_total += loss.item()

        # Output training stats
        if (train_idx % int(train_running_step/2)) == 0:
            print(f'[{epoch}/{EPOCH}], \n\t[Single|Avg] = [{round(loss.item(), 5)}|{round(train_total/(train_idx+1), 5)}]')
    train_mse   = train_mse/train_running_step
    train_ce    = train_ce/train_running_step
    train_total = train_total/train_running_step

    # validation Procedure
    val_running_step = len(val_dataloader)
    print(f'runnnig step: {val_running_step}')

    # Accuracy구현해야됨
    model.eval();
    val_mse   = 0
    val_ce    = 0 
    val_total = 0
    val_acc   = 0
    with torch.no_grad():
        for val_idx, (inputs, labels) in tqdm(enumerate(val_dataloader), total=len(val_dataloader)):
            inputs = inputs.cuda()
            labels_state = labels[:, 1:].cuda()
            labels_behavior = labels[:, 0].type(torch.LongTensor).cuda()
        
            outputs = model(inputs)
            loss_mse = criterionMSE(outputs[:, :-9], labels_state)
            loss_ce = criterionCE(outputs[:, -9:], labels_behavior)
            
            val_mse += loss_mse.item()
            val_ce  += loss_ce.item()
            val_total += val_ce + val_mse
            val_acc += (outputs[:, -9:].argmax(axis=1) == labels_behavior).sum().item()

    val_mse   = val_mse/val_running_step
    val_ce    = val_ce/val_running_step
    val_total = val_total/val_running_step
    val_acc   = val_acc/len(valSet)
    
    if val_total < best_total:
        print(f'\t Best MSE changed [{round(best_total, 5)} ---> {round(val_total, 5)}]')
        save('best', epoch, SAVE_PATH, model, optimizer, is_parallel=IS_PARALLEL)     
        best_total = val_total
    save('last', epoch, SAVE_PATH, model, optimizer, is_parallel=IS_PARALLEL)
    print(f'\t Validaition [Current|Best]: [{round(val_mse, 5)}| {round(best_total, 5)}]')

    res['train_mse'].append(train_mse)
    res['train_ce'].append(train_ce)
    res['train_total'].append(train_total)
    
    res['val_mse'].append(val_mse)
    res['val_ce'].append(val_ce)
    res['val_total'].append(val_total)
    res['val_acc'].append(val_acc)
    
    res['best_total'].append(best_total)
    pd.DataFrame(res).to_csv(SAVE_PATH+'/res.csv', index=False)

runnnig step: 358


  2%|▏         | 8/358 [00:02<09:13,  1.58s/it]

[0/10], 
	[Single|Avg] = [2.52893|2.52893]


 53%|█████▎    | 190/358 [00:05<00:02, 61.25it/s]

[0/10], 
	[Single|Avg] = [0.35316|0.75084]


...

[9/10], 
	[Single|Avg] = [0.13126|0.11513]


100%|██████████| 358/358 [00:05<00:00, 68.55it/s]

runnnig step: 36



100%|██████████| 36/36 [00:00<00:00, 95.69it/s] 

	 Validaition [Current|Best]: [0.02174| 3.06497]


### 결과해석용

In [8]:
res_list = list(train_data.drop(['0_behavior', 'scenario'], axis=1).columns)
res_list_ison_out = [each for each in res_list if not each.find('ison')>0]

In [9]:
print('이걸로 결과 해석하시면 됩니다.')
print(f'{len(res_list_ison_out)}')

이걸로 결과 해석하시면 됩니다.
161
